# Fome Zero Foods - Visão Cidade

Neste notebook vamos explorar o dataset de restaurantes da Fome Zero Foods, respondendo a perguntas de negócios sobre o contexto da visão cidade do dataset.

## 1. Importando bibliotecas e criando Dataframe

### 1.1 Importando bibliotecas

In [ ]:
import pandas as pd

from pathlib import Path

### 1.2 Criando Dataframe

In [ ]:
# caminho do arquivo
path = Path.cwd().parent / 'data' / 'processed' / 'dataset_processed.csv'

try:
    with open(path, 'r', encoding='utf-8') as arquivo:
        df = pd.read_csv(arquivo)
except FileNotFoundError:
    print(f"Arquivo não encontrado: {path}")

In [ ]:
df.head(3)

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,has_online_delivery,is_delivering_now,price_range,aggregate_rating,rating_color,rating_text,votes,country_name,price_type,color_name
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,3,4.6,3F7E00,Excellent,619,Philippines,expensive,darkgreen
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,3,4.6,3F7E00,Excellent,619,Philippines,expensive,darkgreen
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,...,0,0,4,4.7,3F7E00,Excellent,469,Philippines,gourmet,darkgreen


## 2. Perguntas de Negócio

### 2.1 Qual o nome da cidade que possui mais restaurantes registrados?

In [ ]:
cidade = (
    df.groupby('city')['restaurant_id'] # Agrupamento por cidade e calculo de duas coisas ao mesmo tempo na coluna 'restaurant_id':
    .agg(['nunique', 'min']) # 'nunique': Quantidade de restaurantes (para ver quem tem mais); 'min': O menor ID (para o critério de desempate)
    .reset_index()
    .sort_values(by=['nunique', 'min'], ascending=[False, True]) # Ordenar po 'nunique' (Decrescente/False) -> Quem tem mais aparece primeiro; Em caso de empate, ordenamos por 'min' (Crescente/True) -> Quem tem o ID menor vence
    .reset_index(drop=True)
)

print(f"Cidade que possuí mais restaurantes: {cidade.loc[0, 'city']}")

Cidade que possuí mais restaurantes: Pune


### 2.2 Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [ ]:
cidade = (df.loc[df['aggregate_rating'] > 4, ['city', 'aggregate_rating']]
    .groupby('city')
    .count()
    .sort_values(by='aggregate_rating', ascending=False)
    .reset_index()
)

print(f"Cidade com mais restaurantes com nota média acima de '4': {cidade.iloc[0, 0]}")

Cidade com mais restaurantes com nota média acima de '4': Bangalore


### 2.3 Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [ ]:
cidade = (df.loc[df['aggregate_rating'] < 2.5, ['city', 'aggregate_rating']]
    .groupby('city')
    .count()
    .sort_values(by='aggregate_rating', ascending=False)
    .reset_index()
)

print(f"Cidade com mais restaurantes com nota média abaixo de '2.5': {cidade.iloc[0, 0]}")

Cidade com mais restaurantes com nota média abaixo de '2.5': Gangtok


### 2.4 Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [ ]:
cidade = (df.loc[:, ['city', 'average_cost_for_two']]
    .groupby('city')
    .mean()
    .sort_values(by='average_cost_for_two', ascending=False)
    .reset_index()
)

print(f"Cidade com maior valor médio de um prato para dois: {cidade.iloc[0,0]}")

Cidade com maior valor médio de um prato para dois: Jakarta


### 2.5 Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [ ]:
# Analise dinamica retornando uma string com o nome da cidade
cidade = (
    df.groupby('city')[['cuisines', 'restaurant_id']]
    .agg(
        cuisines_nunique=('cuisines', 'nunique'), 
        cuisines_min=('cuisines', 'min'), 
        restaurant_id_nunique=('restaurant_id', 'nunique'), 
        restaurant_id_min=('restaurant_id', 'min')
    ).sort_values(by=['cuisines_nunique', 'restaurant_id_min'], ascending=[False, True])
    .reset_index()
)

print(f"Cidade com maior valor médio de um prato para dois: {cidade.iloc[0,0]}")

Cidade com maior valor médio de um prato para dois: Birmingham


### 2.6 Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [ ]:
cidade = (df.loc[df['has_table_booking'] == 1, ['city', 'has_table_booking']]
    .groupby('city')
    .count()
    .sort_values(by='has_table_booking', ascending=False)
    .reset_index()
)

print(f"Cidade com maior quantidade de restaurantes que fazem reservas: {cidade.loc[0, 'city']}")

Cidade com maior quantidade de restaurantes que fazem reservas: Bangalore


### 2.7 Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [ ]:
cidade = (df.loc[df['is_delivering_now'] == 1, ['city', 'is_delivering_now']]
    .groupby('city')
    .count()
    .sort_values(by='is_delivering_now', ascending=False)
    .reset_index()
)

print(f"Cidade com maior quantidade de restaurantes que fazem entregas: {cidade.loc[0, 'city']}")

Cidade com maior quantidade de restaurantes que fazem entregas: Aurangabad


### 2.8 Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [ ]:

cidade = (df.loc[df['has_online_delivery'] == 1, ['city', 'has_online_delivery']]
    .groupby('city')
    .count()
    .sort_values(by='has_online_delivery', ascending=False)
    .reset_index()
)

print(f"Cidade com maior quantidade de restaurantes que aceitam pedidos online: {cidade.loc[0, 'city']}")

Cidade com maior quantidade de restaurantes que aceitam pedidos online: Abu Dhabi
